In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate as sci
from scipy.optimize import fsolve
import numpy as np

In [ ]:
YTM=5/100
CpnRate=4.5/100
CpnPeriod=0.5
Maturity=8
FV=100
Amortized=True
PrincipalPaymentSchedule = np.array([10, 20, 30, 40])
TimeFromMaturity = np.array([-6.0, -4.0, -2.0, 0])
CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
CashflowTimes = np.flip(CashflowTimes)
CashflowTimes = CashflowTimes.reshape((len(CashflowTimes), 1)); CashflowTimes

array([[0.5],
       [1. ],
       [1.5],
       [2. ],
       [2.5],
       [3. ],
       [3.5],
       [4. ],
       [4.5],
       [5. ],
       [5.5],
       [6. ],
       [6.5],
       [7. ],
       [7.5],
       [8. ]])

In [ ]:
Cashflows = np.zeros_like(CashflowTimes) + CpnRate*CpnPeriod*FV; 
Cashflows[-1] += FV
Cashflows

array([[  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [  2.25],
       [102.25]])

In [ ]:
CashflowTimesfromMaturity = CashflowTimes - Maturity;  CashflowTimesfromMaturity

array([[-7.5],
       [-7. ],
       [-6.5],
       [-6. ],
       [-5.5],
       [-5. ],
       [-4.5],
       [-4. ],
       [-3.5],
       [-3. ],
       [-2.5],
       [-2. ],
       [-1.5],
       [-1. ],
       [-0.5],
       [ 0. ]])

In [ ]:
t = np.hstack((CashflowTimesfromMaturity, CashflowTimes))
import pandas as pd
Cfls_df = pd.DataFrame(t, columns=['CflTimesfromMaturity', 'CflTimes'])
Cfls_df

,CflTimesfromMaturity,CflTimes
0,-7.5,0.5
1,-7.0,1.0
2,-6.5,1.5
3,-6.0,2.0
4,-5.5,2.5
5,-5.0,3.0
6,-4.5,3.5
7,-4.0,4.0
8,-3.5,4.5
9,-3.0,5.0


In [ ]:
ppS = np.hstack((TimeFromMaturity[:,np.newaxis], PrincipalPaymentSchedule[:,np.newaxis]))
PP_df = pd.DataFrame(ppS, columns=['CflTimesfromMaturity', 'PrincipalPayments']); PP_df

,CflTimesfromMaturity,PrincipalPayments
0,-6.0,10.0
1,-4.0,20.0
2,-2.0,30.0
3,0.0,40.0


In [ ]:
Cfls_df = pd.merge(Cfls_df, PP_df, on='CflTimesfromMaturity', how='outer') 

In [ ]:
Cfls_df = Cfls_df.fillna(0);  Cfls_df

,CflTimesfromMaturity,CflTimes,PrincipalPayments
0,-7.5,0.5,0.0
1,-7.0,1.0,0.0
2,-6.5,1.5,0.0
3,-6.0,2.0,10.0
4,-5.5,2.5,0.0
5,-5.0,3.0,0.0
6,-4.5,3.5,0.0
7,-4.0,4.0,20.0
8,-3.5,4.5,0.0
9,-3.0,5.0,0.0


In [ ]:
Cfls_df['OutstandingPrincipal'] = FV - np.cumsum(Cfls_df['PrincipalPayments'])
Cfls_df['OutstandingPrincipal'] = Cfls_df['OutstandingPrincipal'].shift(1) 
Cfls_df.loc[0, 'OutstandingPrincipal'] = Cfls_df.loc[1, 'OutstandingPrincipal'] + Cfls_df.loc[0, 'PrincipalPayments'] 
Cfls_df['Cpns'] = Cfls_df['OutstandingPrincipal']*CpnRate*CpnPeriod; 
Cfls_df['TotalCfls'] = Cfls_df['Cpns'] + Cfls_df['PrincipalPayments']; Cfls_df

,CflTimesfromMaturity,CflTimes,PrincipalPayments,OutstandingPrincipal,Cpns,TotalCfls
0,-7.5,0.5,0.0,100.0,2.250,2.250
1,-7.0,1.0,0.0,100.0,2.250,2.250
2,-6.5,1.5,0.0,100.0,2.250,2.250
3,-6.0,2.0,10.0,100.0,2.250,12.250
4,-5.5,2.5,0.0,90.0,2.025,2.025
5,-5.0,3.0,0.0,90.0,2.025,2.025
6,-4.5,3.5,0.0,90.0,2.025,2.025
7,-4.0,4.0,20.0,90.0,2.025,22.025
8,-3.5,4.5,0.0,70.0,1.575,1.575
9,-3.0,5.0,0.0,70.0,1.575,1.575


In [ ]:
Bond_Price=Cfls_df['TotalCfls']/(1+(YTM/CpnPeriod))**Cfls_df['CflTimes'];Bond_Price.cumsum()[len(Cfls_df['CflTimes'])-1]

77.10210802688619

In [ ]:
def BondCfls(YTM,CpnRate,CpnPeriod, Maturity, FV=100, Amortized=False, PrincipalPaymentSchedule=np.empty(1),
             TimeFromMaturity = np.empty(1)):

    CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
    CashflowTimes = np.flip(CashflowTimes)
    CashflowTimes = CashflowTimes[:, np.newaxis]
    if Amortized:
        CashflowTimesfromMaturity = CashflowTimes - Maturity
        t = np.hstack((CashflowTimesfromMaturity, CashflowTimes))
        Cfls_df = pd.DataFrame(t, columns=['CflTimesfromMaturity', 'CflTimes'])
        ppS = np.hstack((TimeFromMaturity[:,np.newaxis], PrincipalPaymentSchedule[:,np.newaxis]))
        PP_df = pd.DataFrame(ppS, columns=['CflTimesfromMaturity', 'PrincipalPayments'])
        Cfls_df = pd.merge(Cfls_df, PP_df, on='CflTimesfromMaturity', how='outer')
        Cfls_df = Cfls_df.fillna(0)
        Cfls_df['OutstandingPrincipal'] = FV - np.cumsum(Cfls_df['PrincipalPayments'])
        Cfls_df['OutstandingPrincipal'] = Cfls_df['OutstandingPrincipal'].shift(1) 
        Cfls_df.loc[0, 'OutstandingPrincipal'] = Cfls_df.loc[1, 'OutstandingPrincipal'] + Cfls_df.loc[0, 'PrincipalPayments'] 
        Cfls_df['Cpns'] = Cfls_df['OutstandingPrincipal']*CpnRate*CpnPeriod
        Cfls_df['TotalCfls'] = Cfls_df['Cpns'] + Cfls_df['PrincipalPayments']
        AccruedInterest = (CpnPeriod - Cfls_df.loc[0, 'CflTimes'])*CpnRate*Cfls_df.loc[0, 'OutstandingPrincipal']
        df_out = Cfls_df[['CflTimes', 'TotalCfls']]
        df_out['disc_factor']=1/(1+YTM/CpnPeriod)**df_out['CflTimes']
        Dirty_Price=df_out['TotalCfls']@df_out['disc_factor']
        Dirty_Price=Dirty_Price.cumsum()[len(df_out['CflTimes'])-1]
        Bond_Price=Dirty_Price-AccruedInterest
        df_out['McCauley Duration']=df_out['TotalCfls']/(1+(YTM/CpnPeriod)/100)**df_out['CflTimes']
        df_out['TotalD/P']=df_out['McCauley Duration']*df_out['CflTimes']/Bond_Price.sum()
        Duration= df_out['TotalD/P'].sum()/CpnPeriod
        Modified_Duration= Duration/ (1+(YTM/CpnPeriod))
    else:
        PrincipalPaymentSchedule = np.array([10, 20, 30, 40])
        TimeFromMaturity = np.array([-6.0, -4.0, -2.0, 0])
        CashflowTimes = np.arange(Maturity, 0, -CpnPeriod)
        CashflowTimes = np.flip(CashflowTimes)
        CashflowTimes = CashflowTimes.reshape((len(CashflowTimes), 1))
        
        Cashflows = np.zeros_like(CashflowTimes) + CpnRate*CpnPeriod*FV; 
        Cashflows[-1] += FV
        CashflowTimesfromMaturity = CashflowTimes - Maturity
        t = np.hstack((CashflowTimesfromMaturity, CashflowTimes))
        Cfls_df = pd.DataFrame(t, columns=['CflTimesfromMaturity', 'CflTimes'])
        ppS = np.hstack((TimeFromMaturity[:,np.newaxis], PrincipalPaymentSchedule[:,np.newaxis]))
        PP_df = pd.DataFrame(ppS, columns=['CflTimesfromMaturity', 'PrincipalPayments'])
        Cfls_df = pd.merge(Cfls_df, PP_df, on='CflTimesfromMaturity', how='outer') 
        Cfls_df = Cfls_df.fillna(0)
        Cfls_df['OutstandingPrincipal'] = FV - np.cumsum(Cfls_df['PrincipalPayments'])
        Cfls_df['OutstandingPrincipal'] = Cfls_df['OutstandingPrincipal'].shift(1) 
        Cfls_df.loc[0, 'OutstandingPrincipal'] = Cfls_df.loc[1, 'OutstandingPrincipal'] + Cfls_df.loc[0, 'PrincipalPayments'] 
        Cfls_df['Cpns'] = Cfls_df['OutstandingPrincipal']*CpnRate*CpnPeriod
        Cfls_df['TotalCfls'] = Cfls_df['Cpns'] + Cfls_df['PrincipalPayments']
        Bond_Price=Cfls_df['TotalCfls']/(1+(YTM/CpnPeriod))**Cfls_df['CflTimes']
        Bond_Price=Bond_Price.cumsum()[len(Cfls_df['CflTimes'])-1]
        Cfls_df['McCauley Duration']=Cfls_df['TotalCfls']/(1+(YTM/2)/100)**Cfls_df['CflTimes']
        Cfls_df['TotalD/P']=Cfls_df['McCauley Duration']*Cfls_df['CflTimes']/Bond_Price.sum()
        Duration= Cfls_df['TotalD/P'].sum()/2
        Modified_Duration= Duration/ (1+(YTM/2))
    if Cfls_df['Cpns'].sum()==0:
      Dollar_Duration=(Maturity/(1+YTM/CpnPeriod)*(FV*(1/1+(YTM/CpnPeriod))**CpnPeriod*Maturity))
      DV01=Dollar_Duration * 0.0001
    else:
      Cfls_df['disc_factor']=1/(1+CpnRate)**Cfls_df['CflTimes']
      Cfls_df['dollar']=-Cfls_df['CflTimes']/(1+YTM)*Cfls_df['TotalCfls']*Cfls_df['disc_factor']
      Dollar_Duration=Cfls_df['dollar'].cumsum()[Cfls_df['dollar'].size-1]
      DV01=Dollar_Duration * 0.0001
    

    return  Bond_Price,Duration,Modified_Duration,Dollar_Duration,DV01

YTM=5/100
CpnRate=4.5/100
CpnPeriod=0.5
Maturity=8
FV=100
(CpnRate,CpnPeriod, Maturity, FV=100, Amortized=False, PrincipalPaymentSchedule=np.empty(1),
             TimeFromMaturity = np.empty(1)):

In [ ]:
metabliti=BondCfls(5/100,5/100,1,8,100)
metabliti[0]

99.99999999999997

In [ ]:
Cfls_df['disc_factor']=1/(1+CpnRate)**Cfls_df['CflTimes']
Cfls_df['dollar']=-Cfls_df['CflTimes']/(1+YTM)*Cfls_df['TotalCfls']*Cfls_df['disc_factor']
Dollar_Duration=Cfls_df['dollar'].cumsum()[Cfls_df['dollar'].size-1]
Dollar_Duration

-501.93215447707814

In [53]:
def port_bond(Position,CpnRate, Maturity,FV, YTM,CpnFreq,Amortized,PrincipalPaymentSchedule,TimeFromMaturity):
    df = pd.DataFrame(columns=['Bonds','CpnRate', 'Maturity', 'FV', 'YTM', 'CpnFreq', 'Amortized','PrincipalPaymentSchedule', 'TimeFromMaturity','Bond_Price','Duration','Modified_Duration','Dollar_Duration','DV01','MarketValue'])
    for i in range(len(Position)):
      timeformaturity=np.empty(1)
      principalpaymentschedule=np.empty(1)
      x=BondCfls(YTM[i],CpnRate[i],CpnFreq[i],Maturity[i],FV[i],Amortized[i],principalpaymentschedule,timeformaturity)
      
      df2 = pd.DataFrame({
          "Bonds":[Position[i]],
          "CpnRate":[CpnRate[i]],
           "Maturity":[Maturity[i]], 
           "FV":[FV[i]], 
           "YTM":[YTM[i]], 
           "CpnFreq":[CpnFreq[i]], 
           "Amortized":[Amortized[i]],
           "PrincipalPaymentSchedule":[PrincipalPaymentSchedule[i]], 
           "TimeFromMaturity":[TimeFromMaturity[i]],
           "Bond_Price":[x[0]],
           "Duration":[x[1]],
           "Modified_Duration":[x[2]],
           "Dollar_Duration":[x[3]],
           "DV01":[x[4]]
      })
      frames = [df, df2]
      
      df2['MarketValue']=(df2['CpnRate']*100+100)/((1+df2['YTM'])**df2['Maturity'])*100
      dollar_port=df2['Dollar_Duration']@df2['Bonds'].cumsum()
      df = pd.concat(frames)

    Weights=[]
    df.reset_index(inplace = True,drop = True)
    x=df['MarketValue'].cumsum()[df['MarketValue'].size-1]
    Weights=(df['MarketValue']/x).values.tolist()
    df['Weights'] = Weights
    bpv_port=dollar_port / 10000
    dur_port=df['Duration']@df['Weights']
    mod_port=df['Modified_Duration']@df['Weights']
    return dollar_port,bpv_port,dur_port,mod_port




In [54]:
d_df=port_bond([1,2],[4.5/100,4.5/100],[8,8],[100,100],[5/100,5/100],[1,2],[False,False],[np.empty(1),np.empty(1)],[np.empty(1),np.empty(1)])
d_df

(-1031.5632328583238,
 -0.10315632328583238,
 3.4186724976682905,
 3.3352902416276007)

In [56]:
bp2m = pd.read_excel('BondPortfolio2Manage.xlsx')
bp2m['Shares']=100/bp2m['FV']                   
display(bp2m)

,Bonds,CpnRate,Maturity,FV,YTM,CpnFreq,CleanPrices,Amortized,PrincipalPaymentSchedule,TimeFromMaturity,Shares
0,1,0.0300,0.78,100,0.033423,1,99.723043,False,NaN,NaN,1.000000
1,2,0.0400,1.30,200,0.040984,1,99.860241,False,NaN,NaN,0.500000
2,3,0.0500,2.00,250,0.048088,1,100.356465,False,NaN,NaN,0.400000
3,4,0.0200,3.05,300,0.053205,2,90.762891,False,NaN,NaN,0.333333
4,5,0.0250,4.12,210,0.054640,2,89.193009,False,NaN,NaN,0.476190
5,6,0.0400,4.67,300,0.054988,1,93.946283,False,NaN,NaN,0.333333
6,7,0.0375,5.14,120,0.053747,2,92.782108,False,NaN,NaN,0.833333
7,8,0.0450,5.56,80,0.053086,2,96.147414,False,NaN,NaN,1.250000
8,9,0.0395,6.62,230,0.050985,4,93.579870,False,NaN,NaN,0.434783
9,10,0.0450,7.53,160,0.050106,2,96.828165,False,NaN,NaN,0.625000


In [57]:
bp2m.drop(bp2m.tail(1).index,inplace=True)

In [58]:
port_bond(bp2m['Bonds'].values.tolist(),bp2m['CpnRate'].values.tolist(),bp2m['Maturity'].values.tolist(),bp2m['FV'].values.tolist(),bp2m['YTM'].values.tolist(),bp2m['CpnFreq'].values.tolist(),bp2m['Amortized'].values.tolist(),bp2m['PrincipalPaymentSchedule'].values.tolist(),bp2m['TimeFromMaturity'])


(-6062.414097545468,
 -0.6062414097545469,
 1.3624794418199007,
 1.328265654713111)

In [65]:
#DPortfolio+DHedge=0 -> DPortfolio=-DHedge
zero_ytm=5/100
zero_cpn_per=2
zero_fv=100
zero_maturity=6 
BondCfls(5/100,0,2,6,100)

(90.7066819062677, 2.202155281476183, 2.1484441770499347, 3690.0, 0.369)

In [63]:
dhedge=d_df[0]/(zero_maturity/(1+(zero_ytm/zero_cpn_per)))*(1+(zero_ytm/zero_cpn_per))**(zero_maturity*zero_cpn_per)
dhedge

-237.00355165981728

In [64]:
#zerocpn share sold for hedging interest rate risk
short_zero=dhedge/100
short_zero


-2.370035516598173